In [2]:
import pandas as pd

First, read in the imdb data and the giantbomb data as dataframes and get a count

In [110]:
imdb_games = pd.read_csv("imdb_games_db.csv")
giantbomb_games = pd.read_csv("giantbomb_games_db.csv")

In [52]:
print("Total imdb_games: {} \nTotal giantbomb_games: {}".format(imdb_games.shape[0], giantbomb_games.shape[0]))

Total imdb_games: 14435 
Total giantbomb_games: 35117


We'll remove any exact duplicates from both dataframes

In [111]:
imdb_games = imdb_games.drop_duplicates()
giantbomb_games = giantbomb_games.drop_duplicates()
print("Total imdb_games: {} \nTotal giantbomb_games: {}".format(imdb_games.shape[0], giantbomb_games.shape[0]))

Total imdb_games: 14435 
Total giantbomb_games: 35116


In [120]:
giantbomb_games = giantbomb_games.groupby('game_id').agg({'franchise_id': lambda x: list(x), 'name': 'first', 'api_url': 'first'})

In [122]:
duplicate_giantbomb_names = giantbomb_games[giantbomb_games.duplicated("name", keep=False)]
duplicate_giantbomb_names = duplicate_giantbomb_names.sort_values("name")
duplicate_giantbomb_names

,franchise_id,name,api_url
game_id,,,
44486,[3025-2819],!Sqrxz!,https://www.giantbomb.com/api/game/3030-44486/
44487,[3025-2819],!Sqrxz!,https://www.giantbomb.com/api/game/3030-44487/
19235,[3025-5345],Adventure Quest,https://www.giantbomb.com/api/game/3030-19235/
35012,[3025-3423],Adventure Quest,https://www.giantbomb.com/api/game/3030-35012/
35612,"[3025-2439, 3025-781]",Adventures of Lolo,https://www.giantbomb.com/api/game/3030-35612/
...,...,...,...
24384,[3025-56],beatmania,https://www.giantbomb.com/api/game/3030-24384/
20646,[3025-1624],de Blob,https://www.giantbomb.com/api/game/3030-20646/
55079,[3025-1624],de Blob,https://www.giantbomb.com/api/game/3030-55079/


Get list of all game_ids

In [124]:
game_id_list = duplicate_giantbomb_names.index.to_list()

for each item in the list, get the corresponding json file

### All this is just playing around. I'm using multiple dataframes to test it

In [115]:
# group the data by name and count the number of occurrences
grouped = giantbomb_games.groupby(['name']).size().reset_index(name='count')

# select the rows where count > 1, indicating a duplicate name
duplicates = grouped[grouped['count'] > 1]

# select the rows with duplicate name and different game_id
duplicates = giantbomb_games[giantbomb_games['name'].isin(duplicates['name'])].groupby('name').filter(lambda x: x['game_id'].nunique() > 1)

# print the duplicate rows, sorted by name
if len(duplicates) > 0:
    print("Found {} rows with same name but different game_id:".format(len(duplicates)))
    print(duplicates.sort_values(by=['name', 'game_id']))
else:
    print("No rows with same name but different game_id found.")


Found 1529 rows with same name but different game_id:
      franchise_id  game_id                name  \
13626    3025-2819    44486             !Sqrxz!   
13625    3025-2819    44487             !Sqrxz!   
27430    3025-5345    19235     Adventure Quest   
17638    3025-3423    35012     Adventure Quest   
11318    3025-2439     5026  Adventures of Lolo   
...            ...      ...                 ...   
5536     3025-1624    55079             de Blob   
30366     3025-661    21655         pop'n music   
31003     3025-708    21655         pop'n music   
30386     3025-661    28439         pop'n music   
31095     3025-708    28439         pop'n music   

                                              api_url  
13626  https://www.giantbomb.com/api/game/3030-44486/  
13625  https://www.giantbomb.com/api/game/3030-44487/  
27430  https://www.giantbomb.com/api/game/3030-19235/  
17638  https://www.giantbomb.com/api/game/3030-35012/  
11318   https://www.giantbomb.com/api/game/3030-5026/

Next, 

In [109]:
temp2 = temp[temp.duplicated(["name", "index"], keep=False)]
temp2 = temp2.sort_values("name")
temp2

KeyError: Index(['index'], dtype='object')

In [70]:
a = giantbomb_games[giantbomb_games["name"].str.contains('Sqrxz', na=False, flags=re.IGNORECASE, regex=True)]
a

,franchise_id,game_id,name,api_url
13625,3025-2819,44487,!Sqrxz!,https://www.giantbomb.com/api/game/3030-44487/
13626,3025-2819,44486,!Sqrxz!,https://www.giantbomb.com/api/game/3030-44486/
13627,3025-2819,44489,Sqrxz 2,https://www.giantbomb.com/api/game/3030-44489/
13628,3025-2819,44490,Sqrxz III,https://www.giantbomb.com/api/game/3030-44490/
13629,3025-2819,46046,Sqrxz 4: Cold Cash,https://www.giantbomb.com/api/game/3030-46046/


In [7]:
imdb_games.head()

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,82552,tt0084376,videoGame,"MysteryDisc: Murder, Anyone?","MysteryDisc: Murder, Anyone?",0,1982,\N,\N,"Adventure,Crime,Mystery",6.1,37.0
1,102667,tt0105000,videoGame,Night Trap,Night Trap,0,1992,\N,\N,"Adventure,Horror,Mystery",6.2,376.0
2,107391,tt0109865,videoGame,Gabriel Knight: Sins of the Fathers,Gabriel Knight: Sins of the Fathers,0,1993,\N,\N,"Adventure,Drama,Horror",9.1,706.0
3,107783,tt0110267,videoGame,King's Quest VII: The Princeless Bride,King's Quest VII: The Princeless Bride,0,1994,\N,\N,"Adventure,Fantasy",7.6,206.0
4,108416,tt0110909,videoGame,Psychic Detective,Psychic Detective,0,1995,\N,\N,"Adventure,Fantasy,Mystery",8.5,50.0


In [22]:
imdb_games.nunique()

Unnamed: 0        14435
tconst            14435
titleType             1
primaryTitle      14055
originalTitle     14088
isAdult               2
startYear            61
endYear               1
runtimeMinutes       59
genres              380
averageRating        88
numVotes           1685
dtype: int64

In [54]:
giantbomb_games.head(20)

,franchise_id,game_id,name,api_url
0,3025-1,544,Super Mario All-Stars & Super Mario World,https://www.giantbomb.com/api/game/3030-544/
1,3025-1,6649,Super Mario Sunshine,https://www.giantbomb.com/api/game/3030-6649/
2,3025-1,7314,Super Mario Bros. Deluxe,https://www.giantbomb.com/api/game/3030-7314/
3,3025-1,7358,Super Mario RPG: Legend of the Seven Stars,https://www.giantbomb.com/api/game/3030-7358/
4,3025-1,7406,Super Mario Bros. 2,https://www.giantbomb.com/api/game/3030-7406/
5,3025-1,7551,Super Mario World: Super Mario Advance 2,https://www.giantbomb.com/api/game/3030-7551/
6,3025-1,8942,Super Mario 64 DS,https://www.giantbomb.com/api/game/3030-8942/
7,3025-1,12460,Super Mario Bros. / Duck Hunt,https://www.giantbomb.com/api/game/3030-12460/
8,3025-1,12476,Super Mario Strikers,https://www.giantbomb.com/api/game/3030-12476/
9,3025-1,14425,Super Mario Land 2: 6 Golden Coins,https://www.giantbomb.com/api/game/3030-14425/


In [27]:
giantbomb_games.count()

franchise_id    35117
game_id         35117
name            35117
api_url         35117
dtype: int64

In [26]:
giantbomb_games.nunique()

franchise_id     5502
game_id         29208
name            28479
api_url         29208
dtype: int64

In [ ]:
The first thing I'll do is remove any duplicates in the 2 dataframes and check if this impacts the count.

The first thing I am going to do is check for any titles that appear only once in both dataframes and that have matching titles. With this criteria, we can assume that the games are the same. 

In [ ]:
paired_games = imdb_games[imdb_games['primaryTitle'].isin(giantbomb_games['name'])]

In [13]:
matching_rows = imdb_games[imdb_games['primaryTitle'].isin(giantbomb_games['name'])]
matching_rows.count()

Unnamed: 0        7198
tconst            7198
titleType         7198
primaryTitle      7198
originalTitle     7198
isAdult           7198
startYear         7198
endYear           7198
runtimeMinutes    7198
genres            7198
averageRating     7198
numVotes          7198
dtype: int64

In [14]:
merged_df = pd.merge(matching_rows, giantbomb_games, left_on='primaryTitle', right_on='name')

In [15]:
merged_df.head()

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,franchise_id,game_id,name,api_url
0,107391,tt0109865,videoGame,Gabriel Knight: Sins of the Fathers,Gabriel Knight: Sins of the Fathers,0,1993,\N,\N,"Adventure,Drama,Horror",9.1,706.0,3025-638,14075,Gabriel Knight: Sins of the Fathers,https://www.giantbomb.com/api/game/3030-14075/
1,107783,tt0110267,videoGame,King's Quest VII: The Princeless Bride,King's Quest VII: The Princeless Bride,0,1994,\N,\N,"Adventure,Fantasy",7.6,206.0,3025-413,10656,King's Quest VII: The Princeless Bride,https://www.giantbomb.com/api/game/3030-10656/
2,109215,tt0111727,videoGame,Wing Commander III: Heart of the Tiger,Wing Commander III: Heart of the Tiger,0,1994,\N,\N,"Action,Adventure,Sci-Fi",8.8,815.0,3025-212,18483,Wing Commander III: Heart of the Tiger,https://www.giantbomb.com/api/game/3030-18483/
3,112336,tt0114950,videoGame,Wing Commander IV: The Price of Freedom,Wing Commander IV: The Price of Freedom,0,1995,\N,\N,"Action,Adventure,Sci-Fi",8.8,795.0,3025-212,6922,Wing Commander IV: The Price of Freedom,https://www.giantbomb.com/api/game/3030-6922/
4,112518,tt0115141,videoGame,Command & Conquer: Red Alert,Command & Conquer: Red Alert,0,1996,\N,\N,"Action,Sci-Fi,War",8.5,2402.0,3025-98,249,Command & Conquer: Red Alert,https://www.giantbomb.com/api/game/3030-249/


In [16]:
unmatching_rows = imdb_games[~imdb_games['primaryTitle'].isin(giantbomb_games['name'])]

In [17]:
unmatching_rows.count()

Unnamed: 0        7237
tconst            7237
titleType         7237
primaryTitle      7237
originalTitle     7237
isAdult           7237
startYear         7237
endYear           7237
runtimeMinutes    7237
genres            7237
averageRating     7237
numVotes          7237
dtype: int64

In [23]:
try_this = unmatching_rows[unmatching_rows['originalTitle'].isin(giantbomb_games['name'])]
try_this.count()

Unnamed: 0        116
tconst            116
titleType         116
primaryTitle      116
originalTitle     116
isAdult           116
startYear         116
endYear           116
runtimeMinutes    116
genres            116
averageRating     116
numVotes          116
dtype: int64

In [24]:
merged_df2 = pd.merge(try_this, giantbomb_games, left_on='originalTitle', right_on='name')

In [25]:
merged_df2.head()

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,franchise_id,game_id,name,api_url
0,125840,tt0129342,videoGame,Riven: the Sequel to Myst,Riven: The Sequel to Myst,0,1997,\N,\N,"Adventure,Fantasy,Mystery",8.7,513.0,3025-527,16230,Riven: The Sequel to Myst,https://www.giantbomb.com/api/game/3030-16230/
1,179543,tt0186327,videoGame,Tag Team Match: M.U.S.C.L.E,Kinnikuman,0,1986,\N,\N,\N,3.6,24.0,3025-1929,80402,Kinnikuman,https://www.giantbomb.com/api/game/3030-80402/
2,181929,tt0188924,videoGame,Muppets Treasure Island,Muppet Treasure Island,0,1996,\N,\N,"Adventure,Comedy,Family",7.1,800.0,3025-1064,4225,Muppet Treasure Island,https://www.giantbomb.com/api/game/3030-4225/
3,196147,tt0204287,videoGame,Donkey Kong Junior,Donkey Kong Jr.,0,1982,\N,\N,"Action,Family",6.6,306.0,3025-1,19416,Donkey Kong Jr.,https://www.giantbomb.com/api/game/3030-19416/
4,196147,tt0204287,videoGame,Donkey Kong Junior,Donkey Kong Jr.,0,1982,\N,\N,"Action,Family",6.6,306.0,3025-1,27840,Donkey Kong Jr.,https://www.giantbomb.com/api/game/3030-27840/


In [28]:
unmatching_rows

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,82552,tt0084376,videoGame,"MysteryDisc: Murder, Anyone?","MysteryDisc: Murder, Anyone?",0,1982,\N,\N,"Adventure,Crime,Mystery",6.1,37.0
1,102667,tt0105000,videoGame,Night Trap,Night Trap,0,1992,\N,\N,"Adventure,Horror,Mystery",6.2,376.0
4,108416,tt0110909,videoGame,Psychic Detective,Psychic Detective,0,1995,\N,\N,"Adventure,Fantasy,Mystery",8.5,50.0
6,111284,tt0113863,videoGame,Mr. Payback: An Interactive Movie,Mr. Payback: An Interactive Movie,0,1995,\N,30,"Adventure,Comedy,Sci-Fi",4.7,104.0
7,111691,tt0114282,videoGame,Ride for Your Life,Ride for Your Life,0,1995,\N,28,"Adventure,Comedy,Sci-Fi",5.8,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14430,9712195,tt9888864,videoGame,Infernal,Infernal,0,2007,\N,\N,Action,6.0,17.0
14431,9713877,tt9892552,videoGame,Spiral Splatter,Spiral Splatter,0,2017,\N,\N,Action,2.0,10.0
14432,9714234,tt9893348,videoGame,Alien: Covenant In Utero VR Experience,Alien: Covenant In Utero VR Experience,0,2017,\N,\N,Sci-Fi,6.2,17.0
14433,9714437,tt9893804,videoGame,The Lego Movie 2 Videogame,The Lego Movie 2 Videogame,0,2019,\N,\N,"Adventure,Animation,Family",5.5,156.0


In [35]:
import re
temp = giantbomb_games[giantbomb_games["name"].str.contains('Infernal', na=False, flags=re.IGNORECASE, regex=True)]
temp.head(50)

,franchise_id,game_id,name,api_url
18481,3025-358,7284,Indiana Jones and the Infernal Machine,https://www.giantbomb.com/api/game/3030-7284/


My current plan for the most accurate way to approach this:

First check for duplicate names in both datasets. If there is a duplicate name in either dataset, we need to check whether this is because there are 2 games with the same name. If this is the case, we need a way to distinguish between the 2. I would anticipate year of release could work.

The next step would be comparing names against alternate titles. After this, any that still have no matches, would be subject to fuzzy matching.I'd anticipate at least 75% of the games in the imdb dataset should be contained in giantbomb's database.

In [40]:
imdb_dups = imdb_games[imdb_games.duplicated(['primaryTitle', 'startYear'], keep=False)]
imdb_dups = imdb_dups.sort_values("primaryTitle")
print(imdb_dups)

       Unnamed: 0      tconst  titleType  \
12954     8255773   tt6705128  videoGame   
1136       279295   tt0291777  videoGame   
303        196626   tt0204797  videoGame   
304        196627   tt0204798  videoGame   
12289     7854936   tt5809162  videoGame   
...           ...         ...        ...   
11667     7523201   tt5058424  videoGame   
4318       882017   tt0910971  videoGame   
5703      2116024   tt1206584  videoGame   
7398      3835299  tt15244974  videoGame   
569        238738   tt0249252  videoGame   

                                         primaryTitle  \
12954                                    A Bug's Life   
1136                                     A Bug's Life   
303                                           Aladdin   
304                                           Aladdin   
12289                              Alien vs. Predator   
...                                               ...   
11667                                     Tom & Jerry   
4318   Wallace 

In [43]:
imdb_dups.head(50)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
12954,8255773,tt6705128,videoGame,A Bug's Life,A Bug's Life,0,1998,\N,\N,"Adventure,Family",6.2,11.0
1136,279295,tt0291777,videoGame,A Bug's Life,A Bug's Life,0,1998,\N,\N,"Adventure,Comedy,Family",6.6,742.0
303,196626,tt0204797,videoGame,Aladdin,Aladdin,0,1993,\N,\N,"Action,Adventure,Fantasy",7.3,337.0
304,196627,tt0204798,videoGame,Aladdin,Aladdin,0,1993,\N,\N,"Action,Adventure,Animation",7.4,965.0
12289,7854936,tt5809162,videoGame,Alien vs. Predator,Alien vs. Predator,0,1994,\N,\N,"Action,Sci-Fi",7.4,42.0
3395,452529,tt0471072,videoGame,Alien vs. Predator,Alien vs. Predator,0,1994,\N,\N,"Action,Adventure,Sci-Fi",6.8,969.0
10975,6915876,tt3687564,videoGame,Animaniacs,Animaniacs,0,1994,\N,\N,Action,7.6,47.0
13021,8286348,tt6776794,videoGame,Animaniacs,Animaniacs,0,1994,\N,\N,\N,6.3,20.0
5456,1839604,tt11568382,videoGame,Batman,Batman,0,1989,\N,\N,Action,7.6,104.0
317,198298,tt0206583,videoGame,Batman,Batman,0,1989,\N,\N,"Action,Adventure,Crime",7.4,481.0


In [48]:
gb_dups = giantbomb_games[giantbomb_games.duplicated(["name", "game_id", "franchise_id"], keep=False)]
gb_dups = gb_dups.sort_values("name")
gb_dups

,franchise_id,game_id,name,api_url
12948,3025-269,39509,Sega Vintage Collection: ToeJam & Earl,https://www.giantbomb.com/api/game/3030-39509/
12949,3025-269,39509,Sega Vintage Collection: ToeJam & Earl,https://www.giantbomb.com/api/game/3030-39509/
